
# 数据预处理的概念

在深度学习中，模型的输入并不是实际的文字、图片或音频的原始信息。以NLP来说，文字或单词可能会先经过拆分编码转换为字元，再经过一个字典（vocab）转换为一个数字编号，
举个例子，reading可能会被转换为#read和#ing，而后变为字典编号1和19（仅作为例子，其中的数字和字符可能不准确）。以CV来说，图片在输入模型之前可能会经过增强、旋转、裁剪、二值化等过程，形成指定大小和规则的新图片，
这个过程就是预处理。

# ModelScope的预处理器

预处理器在ModelScope中的作用比单纯的tokenizer更宽泛，它开始于数据从数据集中取出，终止于处理好的数据输入模型。因此预处理过程会关心数据集本身特性，比如会在构造方法中传入数据集的key用来获取对应字段的值。

ModelScope的预处理器也是通过注册被自动使用的，它的注册方式是领域+预处理器名字，例如：


In [1]:
# 注册一个NLP领域的fill-mask预处理器
@PREPROCESSORS.register_module(Fields.nlp, module_name=Preprocessors.fill_mask)



如果是外部的预处理器，也可以通过直接调用的方法进行注册：


In [1]:
PREPROCESSORS.register_module(Fields.nlp, module_name=Preprocessors.fill_mask, module_cls=MyPreprocessorCls)



预处理器可以这样使用：


In [1]:
from modelscope.hub.snapshot_download import snapshot_download
from modelscope.preprocessors import SequenceClassificationPreprocessor
from modelscope.models.nlp import SbertForSequenceClassification
model_dir = snapshot_download('damo/nlp_structbert_sentence-similarity_chinese-base')
# 直接构造，这时候注册机制不起作用
preprocessor = SequenceClassificationPreprocessor(model_dir=model_dir, sequence_length=256)
model = SbertForSequenceClassification.from_pretrained(model_dir)
#输入一个tuple
data = preprocessor(('这件商品很好', '这件商品很优秀'))
print(data)
print(model(**data)) # AttentionTextClassificationModelOutput(logits=tensor([[-1.3232,  1.5160]], grad_fn=<AddmmBackward0>), loss=None, attentions=None, hidden_states=None)



也可以调用Preprocessor的from_pretrained方法：


In [1]:
from modelscope.preprocessors import Preprocessor
from modelscope.models import Model
# 在参数中指定了双句的两个key
preprocessor = Preprocessor.from_pretrained('damo/nlp_structbert_sentence-similarity_chinese-base', first_sequence='sent1', second_sequence='sent2')
model = Model.from_pretrained('damo/nlp_structbert_sentence-similarity_chinese-base')
data = preprocessor({'sent1': '这件商品很好', 'sent2': '这件商品很优秀'})
print(data)
print(model(**data)) # AttentionTextClassificationModelOutput(logits=tensor([[-1.3232,  1.5160]], grad_fn=<AddmmBackward0>), loss=None, attentions=None, hidden_states=None)


[注：] 模型可以通过snapshot_download函数下载到本地，也可以是任何一个本地模型目录（代替model_dir即可）。

在上面的例子中，SequenceClassificationPreprocessor这个预处理器做了两件事情：
1. 从一个完整的输入中分离出句子1和句子2
2. 将句子1和句子2传入内部的tokenizer，并生成可以输入模型的数据格式

在不同模型和任务下，预处理器的表现不同，请根据您感兴趣的模态来查看不同部分的文档。


## 在训练中使用预处理器
在训练中使用预处理器的方式请查看[模型的训练](./模型的训练Train.ipynb)。

## 在推理中使用预处理器
在推理中使用预处理器的方式请查看[模型的推理](./模型的推理Pipeline.ipynb)。

## 预处理在文件中的配置
为模型配置预处理器请查看[模型的配置](../开发者使用指南/Configuration详解.ipynb)。

# NLP

NLP模态的预处理器都继承于一个基类[NLPBasePreprocessor]()：


In [1]:
class NLPBasePreprocessor(Preprocessor):

    def __init__(self,
                 model_dir: str,
                 first_sequence=None,
                 second_sequence=None,
                 label=None,
                 label2id=None,
                 mode=ModeKeys.INFERENCE,
                 **kwargs):
    pass



这个类在from_pretrained或构造方法中支持传入：
- first_sequence
- second_sequence
- label
这三个字段用来表示数据集中字段1、字段2、label字段的key值
- label2id 类型为dict，表示数据集的label和id的映射关系，如果不传入会使用配置文件中的值，如果传入会覆盖配置文件的值。

通常来说，这四个参数都和数据集相关，因此用户在使用**任意**NLP预处理器时都可以通过传入这些参数来适配特定数据集。

- mode 预处理器的运行状态是推理(inference)、训练（train）还是校验(eval)。

其他非标准参数在configuration.json或对应子类的构造方法中都有默认值，一般情况下用户可以不用关心。
如果在使用特定模型时需要改动这些参数，您可以查看以下文档：


### SequenceClassificationPreprocessor

这个预处理器基于transformers.tokenizer实现，用于各输入格式符合transformers标准定义的文本分类预处理。



In [1]:
from modelscope.hub.snapshot_download import snapshot_download
from modelscope.preprocessors import SequenceClassificationPreprocessor

model_id = 'damo/nlp_structbert_sentiment-classification_chinese-tiny'
model_dir = snapshot_download(model_id)
sentence = '启动的时候很大声音，然后就会听到1.2秒的卡察的声音，类似齿轮摩擦的声音'

tokenizer = SequenceClassificationPreprocessor(model_dir)
result = tokenizer(sentence)
print (result)



分类任务在推理阶段返回的结果如下示：
```json
{'input_ids': tensor([[ 101, 1423, 1220, 4638, 3198,  952, 2523, 1920, 1898, 7509, 8024, 4197,
         1400, 2218,  833, 1420, 1168,  122,  119,  123, 4907, 4638, 1305, 2175,
         4638, 1898, 7509, 8024, 5102,  849, 7976, 6762, 3040, 3092, 4638, 1898,
         7509,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}
```

这个预处理器的输入参数除支持了NLPBasePreprocessor的参数外，还支持：

- sequence_length 输入字符串的最大长度
- kwargs transformers的tokenizer各参数均可以通过kwargs传入，运行时会传到tokenizer的__call__方法中

在模型的推理阶段，预处理器的mode被设置为`inference`，这时预处理器的返回值类型是Mapping{str: torch.Tensor}，这时用户可以直接将其输入模型中，

在训练或校验阶段，预处理器的mode是`train`或`eval`，预处理器的返回值类型是Mapping{str: List}，这时候需要trainer的collate来将其转换为Tensor再输入模型。

设置预处理器的mode可以通过Preprocessor.from_pretrained方法的preprocessor_mode参数来实现。在训练阶段，如果用户使用了ModelScope的trainer，那么trainer会自动设置该字段。

### TextGenerationPreprocessor

这个预处理模块与文本分类任务类似，不同的是由于生成任务主要是一个auto regression的过程而不关心输入token的类型，因此默认输出没有 `token_type_ids`这个字段。


In [1]:
from modelscope.preprocessors import TextGenerationPreprocessor
from modelscope.hub.snapshot_download import snapshot_download

model_id = 'damo/nlp_gpt3_text-generation_chinese-base'
model_dir = snapshot_download(model_id)
sentence = '我很好奇'

tokenizer = TextGenerationPreprocessor(model_dir)
result = tokenizer(sentence)
print(result)


值得注意的是在训练阶段，输入不仅需要传入原始文本，还要传入续写生成的目标文本，分别用`src_txt`, `tgt_txt`的key，通过字典形式传入预处理方法，相应示例如下：


In [1]:
from modelscope.preprocessors import TextGenerationPreprocessor
from modelscope.utils.constant import ModeKeys
from modelscope.hub.snapshot_download import snapshot_download

model_dir = snapshot_download('damo/nlp_gpt3_text-generation_chinese-base')
src_txt = '我很好奇'
tgt_txt = '这个问题是如何被大众解决的'


tokenizer = TextGenerationPreprocessor(model_dir, mode=ModeKeys.TRAIN)
result = tokenizer({'src_txt':src_txt, 'tgt_txt':tgt_txt})
print(result)



### 序列标注任务预处理
ModelScope序列标注针对的如中文命名实体识别，中文词性标注等任务，文本预处理与分类类似， 具体示例如下：


In [1]:
from modelscope.preprocessors import TokenClassificationPreprocessor
from modelscope.hub.snapshot_download import snapshot_download

model_dir = snapshot_download('damo/nlp_gpt3_text-generation_chinese-base')
sentence = '今天天气不错，适合出去游玩'

tokenizer = TokenClassificationPreprocessor(model_dir)
result = tokenizer(sentence)
print(result)


输出结果为：


In [1]:
‘token':{'input_ids': tensor([[ 101,  791, 1921, 1921, 3698,  679, 7231, 8024, 6844, 1394, 1139, 1343,
         3952, 4381,  102]]), 
        'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 
        'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 
        'text': '今天天气不错，适合出去游玩'
        }


在预处理内部，会尝试将句子拆分为基本字母或文字后再进行tokenize动作，以下伪代码说明了这一点：


In [1]:
sentence = '我是一个机器人'
sentence = sentence.replace(' ', '').strip()
input_tokens = [t for t in sentence] # ['我', '是', '一', '个', '机', '器', '人']
output = tokenize(input_tokens, is_split_into_words=True)
...


如果在推理时有其他需求导致token处理与此不同，用户可以更改输入为dict并将sentence类型改变为token-list，这样Preprocessor就不会对句子进行二次切分，而会直接输入tokenizer：


In [1]:
from modelscope.preprocessors import TokenClassificationPreprocessor
from modelscope.hub.snapshot_download import snapshot_download

model_dir = snapshot_download('damo/nlp_gpt3_text-generation_chinese-base')
# A fake example
sentence = {'first_sequence': ['今天', '天气', '不错，适合出去游玩']}

tokenizer = TokenClassificationPreprocessor(model_dir, first_sequence='first_sequence')
result = tokenizer(sentence)
print(result)


训练阶段TokenClassificationPreprocessor只接受dict输入类型，用户可以在config文件中指定dict key，如：


In [1]:
cfg.preprocessor.first_sequence = 'sentence1'
cfg.preprocessor.label = 'label'


或者也可以像推理时创建好Preprocessor并传入trainer：


In [1]:
from modelscope.preprocessors import TokenClassificationPreprocessor

tokenizer = TokenClassificationPreprocessor(model_dir, 
                                            first_sequence='sentence1',
                                            label='label')


在训练的文件中，请保证first_sequence指代的内容是一个List，其中的每个元素是需要被token化的一个或多个字符，label指代的内容同样是List，其中的每个元素是first_sequence中元素对应的label。
如果用户在训练分词任务，而源数据是空格分开的句子，且没有标注好的label，则可以方便地使用另一个Preprocessor来处理这个数据集：


In [1]:
from modelscope.preprocessors.nlp import WordSegmentationBlankSetToLabelPreprocessor

preprocessor = WordSegmentationBlankSetToLabelPreprocessor()
preprocessor('今天 天气 不错，适合 出去 游玩')


### 完形填空任务预处理
ModelScope也提供了中英文文本Fill Mask完形填空的任务预处理模块，具体使用细节与分类任务类似。具体调用示例如下：


In [1]:
from modelscope.preprocessors import NLPPreprocessor
from modelscope.hub.snapshot_download import snapshot_download

model_dir = snapshot_download('damo/nlp_gpt3_text-generation_chinese-base')
sentence = '段誉轻[MASK]折扇，摇了摇[MASK]，[MASK]道：“你师父是你的[MASK][MASK]，你师父可不是[MASK]的师父。你师父差得动你，你师父可[MASK]不动我。'

tokenizer = NLPPreprocessor(model_dir)
result = tokenizer(sentence)
print(result)


输出结果为：
```json
token: {'input_ids': tensor([[ 101, 3667, 6289, 6768,  103, 2835, 2794, 8024, 3031,  749, 3031,  103,
         8024,  103, 6887, 8038,  100,  872, 2360, 4266, 3221,  872, 4638,  103,
          103, 8024,  872, 2360, 4266, 1377,  679, 3221,  103, 4638, 2360, 4266,
          511,  872, 2360, 4266, 2345, 2533, 1220,  872, 8024,  872, 2360, 4266,
         1377,  103,  679, 1220, 2769,  511,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]), 
        'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 
        'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
       }
```

### 调用huggingface tokenizer预处理
ModelScope作为一个开放的开源平台，在提供自身完整的一套模型处理框架之外，也提供了其他开源框架接入的支持。例如用户可以直接使用huggingface 的tokenizer兼容的方法接口，调用方法如下：


In [1]:
from modelscope.preprocessors import build_preprocessor
from modelscope.utils.constant import Fields,InputFields

sentence = 'Do not meddle in the affairs of wizards, for they are subtle and quick to anger.'

cfg = dict(type='Tokenize', tokenizer_name='bert-base-cased')
preprocessor = build_preprocessor(cfg, Fields.nlp)
input = {InputFields.text: sentence}
result = preprocessor(input)
print(result)


返回结果为：
```json
'token':{'input_ids': [101, 2079, 2025, 19960, 10362, 1999, 1996, 3821, 
                       1997, 16657, 1010, 2005, 2027, 2024, 11259, 1998, 
                       4248, 2000, 4963, 1012, 102], 
         'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                            0, 0, 0, 0, 0, 0, 0], 
         'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                            1, 1, 1, 1, 1, 1, 1]
        }
```

## 图像预处理
### 图像分割任务预处理
ModelScope提供了一个较为通用的图像分割预处理函数，名称为`image-instance-segmentation-preprocessor`，目前支持通过在配置文件中传入该preprocessor，trainer在build阶段就会自动加载该preprocessor，并根据当前Model状态自动切换为`train`或`eval`所对应的预处理。
目前`image-instance-segmentation-preprocessor`提供了常用的分割图像预处理方法，包括`Resize`，`RandomFlip`，`Normalize`，`Pad`等，此处各处理函数借鉴了.ipynbetetcion。下面示例展示了具体preprocessor的配置，其中type指定preprocessor类型为`image-instance-segmentation-preprocessor`，`train`字段表示模型训练时所对应的预处理，`val`字段表示模型推理时所对应的预处理：
```json
"preprocessor": {
    "type": "image-instance-segmentation-preprocessor",
    "train": [
        {
            "type": "LoadImageFromFile"
        },
        {
            "type": "LoadAnnotations",
            "with_bbox": true,
            "with_mask": true
        },
        {
            "type": "Resize",
            "img_scale": [
                [666, 320],
                [666, 400]
            ],
            "multiscale_mode": "range",
            "keep_ratio": true
        },
        {
            "type": "RandomFlip",
            "flip_ratio": 0.5
        },
        {
            "type": "Normalize",
            "mean": [123.675, 116.28, 103.53],
            "std": [58.395, 57.12, 57.375],
            "to_rgb": true
        },
        {
            "type": "Pad",
            "size_divisor": 32
        },
        {
            "type": "DefaultFormatBundle"
        },
        {
            "type": "Collect",
            "keys": ["img", "gt_bboxes", "gt_labels", "gt_masks"],
            "meta_keys": [
                "filename", "ori_filename", "ori_shape",
                "img_shape", "pad_shape", "scale_factor", "flip",
                "flip_direction", "img_norm_cfg", "ann_file",
                "classes"
            ]
        }
    ],
    "val": [
        {
            "type": "LoadImageFromFile"
        },
        {
            "type": "Resize",
            "img_scale": [1333, 800],
            "keep_ratio": true
        },
        {
            "type": "RandomFlip",
            "flip_ratio": 0.0
        },
        {
            "type": "Normalize",
            "mean": [123.675, 116.28, 103.53],
            "std": [58.395, 57.12, 57.375],
            "to_rgb": true
        },
        {
            "type": "Pad",
            "size_divisor": 32
        },
        {
            "type": "ImageToTensor",
            "keys": ["img"]
        },
        {
            "type": "Collect",
            "keys": ["img"],
            "meta_keys": [
                "filename", "ori_filename", "ori_shape",
                "img_shape", "pad_shape", "scale_factor", "flip",
                "flip_direction", "img_norm_cfg", "ann_file",
                "classes"
            ]
        }
    ]
},
```



